In [7]:
path = r"C:\Users\Freun\Desktop\htcv_mgs\data\MGS_data\data"

from pathlib import Path
from PIL import Image


img_paths = list(Path(path).rglob("*.jpg"))
dims = {}
for img_path in img_paths:
    img = Image.open(img_path)
    if img.size not in dims:
        dims[img.size] = 1
    else:
        dims[img.size] += 1
    img.close()

# print("Image sizes and their counts:")
# for size, count in dims.items():
#     print(f"{size}: {count}")
    
print(f"Smalles image: {min(dims.keys())}")

print("Image sizes that are smaller than 224x224 in one or both dimensions:")
for size, count in dims.items():
    if size[0] < 224 or size[1] < 224:
        print(f"{size}: {count}")

Smalles image: (101, 101)
Image sizes that are smaller than 224x224 in one or both dimensions:
(220, 220): 2
(223, 223): 1
(209, 209): 2
(190, 190): 2
(189, 189): 1
(187, 187): 1
(197, 197): 2
(185, 185): 1
(171, 171): 1
(195, 195): 1
(101, 101): 1
(217, 217): 1
(217, 266): 1
(190, 240): 1
(269, 175): 1
(263, 205): 1
(284, 209): 1
(220, 286): 1
(244, 213): 1
(214, 194): 1
(223, 222): 1
(214, 265): 1
(180, 233): 1
(218, 267): 1
(185, 269): 1
(225, 184): 1
(214, 249): 1
(225, 203): 1
(332, 219): 1
(227, 189): 1
(213, 252): 1
(191, 241): 1
(183, 177): 1
(169, 227): 1
(219, 212): 1
(292, 220): 1
(217, 278): 1
(258, 216): 1
(255, 221): 1
(212, 192): 1
(233, 218): 1
(246, 218): 1
(215, 199): 1
(221, 237): 1
(214, 244): 1
(207, 229): 1
(270, 215): 1
(345, 222): 1
(205, 248): 1
(228, 222): 1
(311, 209): 1
(262, 189): 1
(250, 221): 1
(208, 237): 1
(222, 272): 1
(267, 206): 1
(201, 242): 1
(257, 221): 1
(260, 216): 1
(263, 216): 1
(218, 252): 1
(239, 199): 1
(274, 205): 1
(250, 205): 1
(235, 216

In [2]:
# Get normalization statistics for the image dataset 
import numpy as np
from tqdm import tqdm
from pathlib import Path
from PIL import Image

# labeled dataset
mean = np.zeros(3)
std = np.zeros(3)

path = r"C:\Users\Freun\Desktop\htcv_mgs\data\MGS_data\data"
img_paths = list(Path(path).rglob("*.jpg")) + list(Path(path).rglob("*.png"))
for img_path in tqdm(img_paths):
    img_pil = Image.open(img_path)
    img = np.array(img_pil).astype(np.float32) / 255.0
    mean += np.mean(img, axis=(0, 1))
    std += np.std(img, axis=(0, 1))
    img_pil.close()
    
print("Mean: ", mean / len(img_paths))
print("Std: ", std / len(img_paths))


# whole dataset
mean = np.zeros(3)
std = np.zeros(3)

path = r"C:\Users\Freun\Desktop\htcv_mgs\data\BMv3"
img_paths = list(Path(path).rglob("*.jpg")) + list(Path(path).rglob("*.png"))
for img_path in tqdm(img_paths):
    img_pil = Image.open(img_path)
    img = np.array(img_pil).astype(np.float32) / 255.0
    mean += np.mean(img, axis=(0, 1))
    std += np.std(img, axis=(0, 1))
    img_pil.close()

print("Mean: ", mean / len(img_paths))
print("Std: ", std / len(img_paths))

100%|██████████| 3406/3406 [01:07<00:00, 50.62it/s]  


Mean:  [0.49035715 0.43031338 0.36964953]
Std:  [0.19827825 0.18251361 0.16136015]


100%|██████████| 34939/34939 [35:59<00:00, 16.18it/s]  

Mean:  [0.48938244 0.37434759 0.25759417]
Std:  [0.23947592 0.19973618 0.14904125]


| Kanal | Mittelwert (Mean) | Standardabweichung (Std) |
|-------|-------------------|---------------------------|
| R     | 0.49035715        | 0.19827825                |
| G     | 0.43031338        | 0.18251361                |
| B     | 0.36964953        | 0.16136015                |


In [15]:
import time
import torch
import torchvision.transforms as T
from PIL import Image
import numpy as np
from pathlib import Path
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

# =============== Pfad zu Bildordner ===============
image_folder = r"C:\Users\Freun\Desktop\htcv_mgs\data\MGS_data\data"
image_paths = list(Path(image_folder).rglob("*.jpg"))

# =============== Transformationen vorbereiten ===============
to_tensor = T.ToTensor()
resize_224 = {
    "bicubic": T.Resize((224, 224), interpolation=Image.BICUBIC),
    "bilinear": T.Resize((224, 224), interpolation=Image.BILINEAR),
    "nearest": T.Resize((224, 224), interpolation=Image.NEAREST)
}

# =============== Methoden ===============
def method_bicubic(img): return resize_224["bicubic"](img)
def method_bilinear(img): return resize_224["bilinear"](img)
def method_nearest(img): return resize_224["nearest"](img)

def method_stride(img_tensor):
    cropped = center_crop_to_square_multiple(img_tensor, multiple=224)
    if cropped is None:
        return None
    factor = cropped.shape[1] // 224
    return cropped[:, ::factor, ::factor]

def center_crop_to_square_multiple(img_tensor, multiple=224):
    _, h, w = img_tensor.shape
    min_dim = min(h, w)
    target_size = (min_dim // multiple) * multiple
    if target_size < multiple:
        return None
    top = (h - target_size) // 2
    left = (w - target_size) // 2
    return img_tensor[:, top:top+target_size, left:left+target_size]

def to_numpy(img):
    if isinstance(img, Image.Image):
        img = np.array(img)
    elif isinstance(img, torch.Tensor):
        img = img.detach().cpu()
        if img.max() <= 1.0:
            img = img * 255
        img = img.permute(1, 2, 0).numpy()
    return np.clip(img, 0, 255).astype(np.uint8)

def compare_metrics(ref, other):
    return psnr(ref, other), ssim(ref, other, channel_axis=-1)

# =============== Metrik-Tracking ===============
times = {"bicubic": [], "bilinear": [], "stride": [], "nearest": []}
psnr_list = {"bilinear": [], "stride": [], "nearest": []}
ssim_list = {"bilinear": [], "stride": [], "nearest": []}

# =============== Verarbeitungsschleife ===============
print(f"Starte Verarbeitung von {len(image_paths)} Bildern...\n")
for path in tqdm(image_paths):
    try:
        img = Image.open(path).convert("RGB")
        img_tensor = to_tensor(img)

        # Methode 1 – Bicubic (Referenz)
        t0 = time.time()
        ref_out = method_bicubic(img)
        t1 = time.time()
        times["bicubic"].append((t1 - t0) * 1000)
        ref_np = to_numpy(ref_out)

        # Methode 2 – Bilinear
        t0 = time.time()
        out_bil = method_bilinear(img)
        t1 = time.time()
        times["bilinear"].append((t1 - t0) * 1000)
        psnr_val, ssim_val = compare_metrics(ref_np, to_numpy(out_bil))
        psnr_list["bilinear"].append(psnr_val)
        ssim_list["bilinear"].append(ssim_val)

        # Methode 3 – Stride
        t0 = time.time()
        out_stride = method_stride(img_tensor)
        t1 = time.time()
        if out_stride is None:
            print(f"Bild {path.name} zu klein für Stride.")
            continue
        times["stride"].append((t1 - t0) * 1000)
        psnr_val, ssim_val = compare_metrics(ref_np, to_numpy(out_stride))
        psnr_list["stride"].append(psnr_val)
        ssim_list["stride"].append(ssim_val)

        # Methode 4 – Nearest
        t0 = time.time()
        out_near = method_nearest(img)
        t1 = time.time()
        times["nearest"].append((t1 - t0) * 1000)
        psnr_val, ssim_val = compare_metrics(ref_np, to_numpy(out_near))
        psnr_list["nearest"].append(psnr_val)
        ssim_list["nearest"].append(ssim_val)

    except Exception as e:
        print(f"Fehler bei {path}: {e}")

# =============== Ergebnisse ausgeben ===============
def summarize(name, values):
    values = np.array(values)
    return f"{name:<15}: Mittelwert = {values.mean():.4f}, Std = {values.std():.4f}"

print("\n--- Benchmark-Ergebnisse ---")
for key in times:
    print(summarize(f"Zeit {key}", times[key]))

print()
for key in psnr_list:
    print(summarize(f"PSNR {key}", psnr_list[key]))
    print(summarize(f"SSIM {key}", ssim_list[key]))


Starte Verarbeitung von 3406 Bildern...



 39%|███▉      | 1334/3406 [00:40<00:41, 49.97it/s]

Bild aw_20201009_130446211_mouse_l_06576.jpg zu klein für Stride.


 41%|████      | 1393/3406 [00:42<00:39, 50.39it/s]

Bild aw_20201027_091716929_mouse_l_03719.jpg zu klein für Stride.


...

Bild lw_3-2b.JPG zu klein für Stride.


 75%|███████▌  | 2564/3406 [01:05<00:15, 53.23it/s]

Bild lw_3-6b.JPG zu klein für Stride.


100%|██████████| 3406/3406 [01:30<00:00, 37.53it/s]


--- Benchmark-Ergebnisse ---
Zeit bicubic   : Mittelwert = 2.7382, Std = 2.3283
Zeit bilinear  : Mittelwert = 1.3936, Std = 1.2276
Zeit stride    : Mittelwert = 0.0558, Std = 0.2297
Zeit nearest   : Mittelwert = 0.1169, Std = 0.3198

PSNR bilinear  : Mittelwert = 50.8343, Std = 2.2208
SSIM bilinear  : Mittelwert = 0.9966, Std = 0.0008
PSNR stride    : Mittelwert = 19.9579, Std = 5.8874
SSIM stride    : Mittelwert = 0.5692, Std = 0.1375
PSNR nearest   : Mittelwert = 40.9852, Std = 2.3799
SSIM nearest   : Mittelwert = 0.9675, Std = 0.0157


### Benchmark-Ergebnisse
Bicubic resized images were taken as benchmark results and the resized versions of the other interpolation techniques are compared via SSIM and PSNR with the bicubic result

| Methode   | Zeit (Mittelwert) | Zeit (Std) | PSNR (Mittelwert) | PSNR (Std) | SSIM (Mittelwert) | SSIM (Std) |
|-----------|-------------------|------------|-------------------|------------|-------------------|------------|
| bicubic   | 2.7382            | 2.3283     | -                 | –          | –                 | –          |   
| bilinear  | 1.3936            | 1.2276     | 50.8343           | 2.2208     | 0.9966            | 0.0008     |
| stride    | 0.0558            | 0.2297     | 19.9579           | 5.8874     | 0.5692            | 0.1375     |
| nearest   | 0.1169            | 0.3198     | 40.9852           | 2.3799     | 0.9675            | 0.0157     |


- Stride is much faster than the other two but results large differences in terms of pixel values (psnr) and also large losses in terms of structural similarity (meaning the visible noise and loss of structural informations that a human would also be able to see)
- Nearest Interpolation results in nearly the same loss then Bilinear Interpolation meaning that the images tend to have few texture, are mainly centered and have no hard lines or edge which the nearest interpolation would distort. Therefor for our image dataset the nearest interpolation technique seems the by the best traid of in terms of speed and interpolation accuracy. The results of the model have to tested after the hyperparameters are found.